# Collecting Real Data

Now it's time to actually collect some data!

You've been given a small ESP32-S2 microcontroller and a "shield" or "hat" board that can be attached to the microcontroller. If the boards are not already attached you can connect them. Make sure the 10-13 and 15-17 pins match between the two boards. See the correct alignment in the image below:

![Alignment](imgs/alignment.png)

Once these are connected, attach the microcontroller to your computer. You should see an external drive mount itself with the name "CIRCUITPY" as shown on Windows, and MacOS in Fig.&nbsp;1:

|Mac|Windows|
|:-|:-|
|![MacOS Mounted Filesystem](imgs/maccpymount.png)|![Windows Mounted Filesystem](imgs/wincpymount.png)|
<td colspan=2>Mounted Filesystem for Mac and Windows</td>

To program the microcontroller you'll need to download an editor. I've been using 
[Thonny](https://thonny.org/), which I recommend, but there are others as well. For the purpose of this workshop, let's use Thonny so we're all on the same page. You'll be keeping your microcontroller, so you can experiment with others later if you like.



First download Thonny from [thonny.org](https://thonny.org/) and install it. When you first run Thonny you may see something like this:

|Thonny setup|
|:-|
|![Thonny Setup](imgs/thonny-setup.png)|
|Thonny Setup|

Chose "Standard" initial settings.

When Thonny is finished launching you should see a screen like this. In the lower right hand corner select the menu button and choose "CircuitPython (generic)":

|Thonny Circuitpython Configuration|
|:-|
|![Thonny Setup](imgs/thonny-initial-screen.png)|
|Thonny Configuration|




To control the output pins of the microcontroller and measure voltages we'll also be using python, but it's a slightly different version of python than the one we're using to analyze the data. This version runs on the microcontroller, and it's called [CircuitPython](https://circuitpython.org/). In addition to standard python syntax, and some standard python libraries, it also includes libraries that enable one to access the pins of the microcontroller to control and measure voltages, or to communicate with standard protocols (e.g., $i^2c$, `SPI`, etc.)

For example there is library called `analogio` that includes methods to read voltage values using analog to digital converters (ADCs) on the microcontroller. The ESP32-S2 also has two digital to analog converters (DACs) that can be used to control voltages as well. The DACs on the ESP32-S2 are eight bit DACs, so they have 255 different output levels. The voltage range is very roughly 0-3.3V. The ADCs on the ESP32-S2 are 12-bit devices, but they have a more limited voltage range. They can only measure up to about 2.5V.

The circuit we'll be using in all our experiments has the schematic below:

|Schematic|
|:-|
|![Fig. 2: PCB Schematic](imgs/schematic.png)|
|PCB Microcontroller Schematic|





On the ESP32-S2 pins 10, 13, and 15 can be used to measure analog voltages using ADCs, while pin 17 can be used either as a DAC output, or a digital (binary HIGH/LOW) output.

# Experiment 1: Ohm's Law

This experiment uses the approach described in [Modeling and Visualizing](ModelingVisualizing.ipynb) to "measure" the value of R3 using Ohm's Law. Of course we *know* it's a 2K resistor from the schematic, but you could frame this as an unknown for your students.

To measure a voltage from a particular pin we need to use the `board` library to get a pin object, and the `analogio` library to construct analog input objects. Each analog input object corresponds to a specific pin on the microcontroller that is identified using the board library as shown below:

    import analogio as aio
    import board

    a15 = aio.AnalogIn(board.IO15)
    a10 = aio.AnalogIn(board.IO10)

Then to actually carry out a measurement we need to access the `value` property of the analog input object for any particular pin. For example the following statment would save the current value of the a15 analog input object in a variable name v15:

    v15 = a15.value

To control the output of a DAC we need to create an analog output object:

    dac = aio.AnalogOut(board.IO17)

In order to set the voltage on the pin one can set the `value` propery like so:

    dac.value = 0  # set dac low

So the plan is pretty simple. Write a loop to set the voltage on pin-17, and measure the voltages on pin-15 and pin-10.

1. Establish an empty list to contain the results of the measurements.
2. Set up a loop to iterate over output voltages.
3. For each iteration
    * set the voltage on pin-17
    * wait a bit for the circuit to settle (note the capacitor)
    * measure the voltages on pins 10 and 13
    * save the values in the list after each measurement




Before we let loose and start collecting data there's one additional aspect we need to discuss. CircuitPython runs on over 500 different microcontrollers with many different features. In order for code that's written on one device to have a chance of working correctly on another device the ADCs and DACs don't use the raw input and output values, but rather they use scaled values with a minimum of zero and a maximum of an unsigned 16-bit integer ($2^{16}-1$) or 65535. This means we need to scale the values we read from ADCs and write to the DACs accordingly. Here's an example showing one way to do this. We'll go over this example in detail during the workshop.

    import analogio as aio
    import board
    import time

    ADCMAX = 2**16-1
    DACREF = 3.3

    adcTest = aio.AnalogIn(board.IO15)  # create an ADC object for pin 15

    adcfactor = adcTest.reference_voltage/ADCMAX # factor to convert values to volts
    dacfactor = ADCMAX/3.3

    dac = aio.AnalogOut(board.IO17)

    header = "vout,vtest,time"

    data = []

    dac.value = 0  # set dac low

    print("sleeping.... let C discharge")
    print("3")
    time.sleep(1)
    print("2")
    time.sleep(1)
    print("1")
    time.sleep(1)
    print("0 --- go!")

    t0 = time.monotonic_ns()

    for i in range(0,20):
        vout = i*3.3/20 # compute a voltage based on the loop count
        dac.value = int(dacfactor*vout)
        time.sleep(0.1)
        value = adcTest.value
        data.append((vout, value, time.monotonic_ns()-t0))
        print("vout: ", vout, "raw value:", value)

    print("done!")

    dac.value = 0 # give the LED a rest....

    print(header)

    for i in range(len(data)):
        vout, val, t = data[i] # get data for ith iteration
        print(f"{vout},{val*adcfactor},{t/1e9}")    


Note that we take some pains to print the data in such a way that it's already in "CSV" format, so we can just copy/paste into a csv file and analyze that on the computer.

## Now it's your turn!

Now modify this program to measure the voltage at the junction between the two resistors as well. Be sure to keep the `time.sleep(0.1)` in the loop so the capacitor get's time to charge between measurements. (Why do we need this?)

Once you have the data for this experiment you can jump to the [DataAnalysis](DataAnalysis.ipynb) notebook to analyze that data. Then come back here to learn about the next two experiments.


# Experiment 2: RC Discharge

The next experiment is to measure the RC time constant of an RC circuit.

This could theoretically be done with the DAC in the same way, but it turns out the DAC is not great at sinking current, and it also has a non-neglible output impedance, so for this experiment we'll configure pin-17 as a digital output instead.

## Digital Output

Sometimes (i.e., now!) it's also useful to control a pin as a digital input or output. Here's an example that shows how that can be done. It blinks the LED using pin 17 set up as a digital output pin:

    print("Blinking with digital IO!")

    import digitalio as dio
    import board
    import time

    dout = dio.DigitalInOut(board.IO17)
    dout.direction = dio.Direction.OUTPUT


    while True:
        dout.value = 1
        time.sleep(0.2)
        dout.value = 0
        time.sleep(0.2)
    
Note that when a pin is configured as a digital output (or input) a valid `value` is either `0`(low, 0V) or `1` (high, 3.3V) only.

## Measure the Discharge

Now, make a copy of your program, and modify it to measure the discharge curve of the capacitor. Configure pin-17 as a digital output (as descrbied above). Bring the pin high, then wait 3 seconds, then bring the pin low, and record the start time.

Modify the loop to record the voltage on the capacitor as a function of time. Be sure to take out, or comment out, the `time.sleep()` statement since this time we want to collect data as quickly as possible.

Once you have the data, head back to the [DataAnalysis](DataAnalysis.ipynb) notebook to analyze the data.


# Experiment 3: LED I-V Curve

The last experiment is the characterize the I-V curve of an LED. For the voltages and currents we're using today, the Shockley equation works pretty well (let me know if you're interested in a physically motivated modification that also works well at higher current levels).

The Shockley equation is:

\begin{equation}
I = I_s \left ( e^{\frac{q V}{\eta k_b T}} - 1 \right)
\end{equation}

At the current levels we'll be using we can ignore the `-1` so the equation simplifies to:

\begin{equation}
I = I_s e^{\frac{q V}{\eta k_b T}}
\end{equation}

Now, $I_s$, $q$, $k_b$, and $\eta$ are all constants, so this can be thought of as simply:

\begin{equation}
I = A e^{B V}
\end{equation}

Where $A$ and $B$ are unknown parameters that we'd like to find.

Make a copy of your program and modify it to collect data from your circuit to measure these parameters. Then head back over to the [DataAnalysis](DataAnalysis.ipynb) notebook to analyze!